Read the buffers file

In [1]:
import geopandas as gpd
import pathlib

In [133]:
path = pathlib.Path(r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\PEP725\After_2016_sent_from_PEP725\pep725_outputs\PEP725_buffers.geojson")
buffers = gpd.read_file(path).set_crs(32632, inplace=True, allow_override=True)

# Create envelopes for the buffers
envelope_series = buffers.geometry.envelope
envelope_series.rename('envelope_geometry', inplace=True)
envelope_gdf = buffers.merge(envelope_series, left_index=True, right_index=True)
envelope_gdf = envelope_gdf.drop(['geometry'], axis=1).set_geometry('envelope_geometry').rename_geometry('geometry')

# Change the envelope to a list to use it later
envelope_list = envelope_gdf.geometry.tolist()
# Creating a list of tuples that will be used to preserve the indexing information of the GeoDataFrame.
# This may be of use later, to get information from the GeoDataFrame and put it in the image, e.g., a label such as the class (DBL, EC, M).
envelope_list_with_index = []
for index, row in envelope_gdf.iterrows():
    envelope_list_with_index.append((index, row['geometry'], row['s_id']))

In [132]:
print(envelope_list_with_index[3])

(3, <shapely.geometry.polygon.Polygon object at 0x00000271E49F49D0>)


In [15]:
envelope_gdf.head()

,s_id,lon,lat,alt,alt_dem,gss_id,genus,species,phase_id,year,day,date,Label,geometry
0,5363,13.91670,54.0833,2,0,1050100,Alnus,Alnus glutinosa,60,2017,27,2017-01-27,DBL,"POLYGON ((817520.468 5999973.720, 825520.468 5..."
1,1554,7.51667,51.7333,60,72,2210500,Salix,Salix caprea,60,2017,29,2017-01-29,DBL,"POLYGON ((393567.206 5728416.903, 401567.206 5..."
2,3120,8.68333,49.5500,140,261,1050100,Alnus,Alnus glutinosa,60,2017,31,2017-01-31,DBL,"POLYGON ((473094.080 5484647.767, 481094.080 5..."
3,2021,8.58333,50.0000,100,101,1050100,Alnus,Alnus glutinosa,60,2017,32,2017-02-01,DBL,"POLYGON ((466138.525 5534713.881, 474138.525 5..."
4,1521,7.83333,51.7000,60,58,1050100,Alnus,Alnus glutinosa,60,2017,33,2017-02-02,DBL,"POLYGON ((415374.473 5724316.443, 423374.473 5..."


In [100]:
import glob
from zipfile import ZipFile
import fnmatch
import datetime

sentinel_2_directory = r"C:\Users\Kostas\Desktop\GIMA\Module_7\Data\Sentinel2_images"

# Create a list of all the Sentinel-2 zipfiles
sentinel_2_zip_list = glob.glob(str(sentinel_2_directory) + '/*.zip', recursive=True)

# Get the date from the Sentinel-2 zip archive. Returns a datetime class object
def getS2Date(s2zip_path):
    string_parts = s2zip_path.split("_")
    band_string = string_parts[-1]
    band_string = band_string.replace('.zip','')
    string_parts = band_string.split("T")
    s2_date = string_parts[0]
    s2_time = string_parts[1]
    s2_date = datetime.datetime.strptime(s2_date, "%Y%m%d").date()
    #print(band_string)
    #print(s2_date, s2_time)
    return s2_date#, s2_time This is in case you want the tie as well. It will be returned as a tuple

for s2zip_path in sentinel_2_zip_list:
    s2date = getS2Date(s2zip_path)
    print(s2date)



# Example output: 20170420T103021_B02_10m



2017-04-20
2022-10-22
2020-09-21


In [68]:
# Create lists from the GeoDataFrame features to use

stations_list = envelope_gdf['s_id'].tolist()
dates_list = envelope_gdf['date'].tolist()
veg_classes_list = envelope_gdf['Label'].tolist()

In [ ]:
#from datetime import datetime, timedelta
import datetime
# Returns a list of dates inbetween start and end date in the format YYYY-MM-DD
def date_range(start_date, end_date):
    current_date = start_date
    while current_date <= end_date:
        yield current_date
        current_date += timedelta(days=1)
    
dates = [date.strftime("%Y-%m-%d") for date in date_range(date_start, date_end)]
print(dates)


In [129]:
def get_data(station, vegetation_class, start_date, end_date):
    data = [station, ]

def calc_target_highest_freq(data):
    pass


stations = []
veg_classes = []
for i_id in range(0, len(sentinel_2_zip_list) - 1):
    image_1 = sentinel_2_zip_list[i_id]
    image_2 = sentinel_2_zip_list[i_id + 1]

    date_start = getS2Date(image_1)
    date_end = getS2Date(image_2)
    dates = [date.strftime("%Y-%m-%d") for date in date_range(date_start, date_end)]
    print(f'Timeslot {i_id+1}:', date_start, date_end)
    data_list = []
    for s in range(0, len(stations_list)):
            for d in range(0, len(dates)):
                if dates[d] == dates_list[d]:
                    data_list = [stations_list[s], veg_classes_list[s], dates_list[s]]
                    print(data_list)
            # data = get_data(s, v, date_start, date_end)
            # target = calc_target_highest_freq(data)

Timeslot 1: 2017-04-20 2022-10-22
Timeslot 2: 2022-10-22 2020-09-21


In [116]:
image_1 = sentinel_2_zip_list[0]
image_2 = sentinel_2_zip_list[1]

date_start = getS2Date(image_1)
date_end = getS2Date(image_2)
print(date_start, date_end)
date_range(date_start, date_end)


2017-04-20 2022-10-22


<generator object date_range at 0x00000271EA2C9930>

In [78]:
date_start = stringifyDate(getS2Date(sentinel_2_zip_list[0]))
date_end = stringifyDate(getS2Date(sentinel_2_zip_list[1]))

a = dates_list[0]
a = datetime.datetime.strptime(a, "%Y-%m-%d").date()
print(a)
print(type(a))

2017-01-27
<class 'datetime.date'>
